# Assignment 3

Sven Den Hartog, s1003026 

Denise Klep, s4210646

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.dataset import concat_examples
from chainer.datasets import TupleDataset
from chainer.cuda import to_cpu
from chainer import serializers
import matplotlib.pyplot as plt
import utils

In [2]:
# create toy data - compute sum of the previous and current input
def create_data(n=3000):

    X = np.random.rand(n,1).astype('float32')
    T = np.sum(np.hstack((X[0:-1],X[1:])),axis=1)
    T = np.hstack([0, T[0:]]).astype('float32')
    T = T.reshape([n,1])

    return TupleDataset(X, T)

In [3]:
max_epoch = 20
batchsize = 32
class ParallelSequentialIterator(chainer.dataset.Iterator):
    def __init__(self, dataset, batch_size, repeat=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.epoch = 0
        self.is_new_epoch = False
        self.repeat = repeat
        self.offsets = [i * len(dataset) // batch_size for i in range(batch_size)]
        self.iteration = 0

    def __next__(self):
        length = len(self.dataset)
        if not self.repeat and self.iteration * self.batch_size >= length:
            raise StopIteration
        cur_words = self.get_words()
        self.iteration += 1
        next_words = self.get_words()

        epoch = self.iteration * self.batch_size // length
        self.is_new_epoch = self.epoch < epoch
        if self.is_new_epoch:
            self.epoch = epoch

        return list(zip(cur_words, next_words))

    @property
    def epoch_detail(self):
        return self.iteration * self.batch_size / len(self.dataset)

    def get_words(self):
        return [self.dataset[(offset + self.iteration) % len(self.dataset)]
                for offset in self.offsets]

    def serialize(self, serializer):
        self.iteration = serializer('iteration', self.iteration)
        self.epoch = serializer('epoch', self.epoch)

train_iter = ParallelSequentialIterator(create_data(), batchsize)
test_iter = ParallelSequentialIterator(create_data(), 1, repeat=False)

In [10]:
class RNN(Chain):
    def __init__(self):
        super(RNN, self).__init__()
        with self.init_scope():
            self.lstm = L.LSTM(None, 50)  # the first LSTM layer
            self.out = L.Linear(None, 1)  # the feed-forward output layer

    def reset_state(self):
        self.lstm.reset_state()

    def __call__(self, x):
        h = self.lstm(x)
        y = self.out(h)
        return y

In [11]:
model = RNN()
optimizer = optimizers.SGD()
optimizer.setup(model)

In [25]:
mean_acc = [] 
mean_test_loss = [] 
mean_train_loss = []
train_losses = []
while train_iter.epoch < max_epoch:
    if train_iter.is_new_epoch:
        rnn.reset_state()
    train_batch = train_iter.next()
    input_train, output_train = concat_examples(train_batch) 

    prediction_train = model(input_train)
    print(output_train)
    
    loss += F.mean_squared_error(prediction_train, output_train)
    if train_iter.iteration % 30 == 0 or train_iter.iteration == seqlen:
        train_losses.append(loss.data)
        model.cleargrads()
        loss.backward()
        loss.unchain_backward()
        optimizer.update()
    
    if train_iter.is_new_epoch:

        test_losses = []
        test_accuracies = []
        while True:
            test_batch = test_iter.next()
            input_test, output_test = concat_examples(test_batch) 

            prediction_test = model(image_test)

            loss_test = F.softmax_cross_entropy(prediction_test, target_test)
            test_losses.append(to_cpu(loss_test.data))

            accuracy = F.accuracy(prediction_test, target_test)
            accuracy.to_cpu()
            test_accuracies.append(accuracy.data)

            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break
                #get out of test loop if test epoch completed

        mean_acc.append(np.mean(test_accuracies))
        mean_test_loss.append(np.mean(test_losses))
        mean_train_loss.append(np.mean(train_losses))
        #track mean losses for visualization later
        train_losses = []

[((array([ 0.99651617], dtype=float32), array([ 1.01460695], dtype=float32)), (array([ 0.54187912], dtype=float32), array([ 1.53839529], dtype=float32))), ((array([ 0.92200267], dtype=float32), array([ 0.92783344], dtype=float32)), (array([ 0.56279361], dtype=float32), array([ 1.48479629], dtype=float32))), ((array([ 0.20945838], dtype=float32), array([ 0.30626774], dtype=float32)), (array([ 0.85213119], dtype=float32), array([ 1.0615896], dtype=float32))), ((array([ 0.56855834], dtype=float32), array([ 0.91466272], dtype=float32)), (array([ 0.7292186], dtype=float32), array([ 1.29777694], dtype=float32))), ((array([ 0.00968447], dtype=float32), array([ 0.55245632], dtype=float32)), (array([ 0.45579591], dtype=float32), array([ 0.46548039], dtype=float32))), ((array([ 0.54328036], dtype=float32), array([ 1.05752254], dtype=float32)), (array([ 0.78159422], dtype=float32), array([ 1.32487464], dtype=float32))), ((array([ 0.56461245], dtype=float32), array([ 1.16558111], dtype=float32)), 

InvalidType: 
Invalid operation is performed in: MeanSquaredError (Forward)

Expect: in_types[0].shape == in_types[1].shape
Actual: (32, 1) != (32, 2, 1)